# Prérequis

Premièrement, nous avons besoin d'importer les bibliothèques python nécessaires pour notre algorithme.

La bibliothèque la plus importante est py_entitymatching, qui est développée par Anhais Group dans le cadre du projet Magellan.

https://sites.google.com/site/anhaidgroup/home

In [ ]:
# importation des bibliothèques requises (notamment py_entitymatching)

!pip install unidecode
!pip install fuzzywuzzy
!pip install missingno
!pip uninstall Cython -y
!pip uninstall py_entitymatching -y
!pip uninstall py_stringsimjoin -y
!pip install py_stringsimjoin==0.3.3
!pip install py_entitymatching==0.4.1


import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np
from fuzzywuzzy import fuzz
from unidecode import unidecode
import missingno as msno
import py_stringsimjoin as ssj
import py_stringmatching as sm
import py_entitymatching as em
import math

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.2 MB/s eta 0:00:00
Found existing installation: Cython 3.0.8
Uninstalling Cython-3.0.8:
  Successfully uninstalled Cython-3.0.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for py_stringsimjoin: filename=py_stringsimjoin-0.3.3-cp310-cp310-linux_x86_64.whl size=4517620 sha256=7ee9afefd323f3166ee71df6f912b78b43ddfdd6a7abc1d8388cd77cd0e73c79
  Stored in directory: /root/.cache/pip/wheels/63/47/e6/f221c81b7b7da486f12483e0e1e766e8be3b1c25e1f08a9203
  Created wheel for py_stringmatching: filename=py_stringmatching-0.4.5-cp310-cp310-linux_x86_64.whl size=2834010 sha256=0a64745948ec5e753161c0ce0cad5fe79f5d4bc6e4b0800236638eb6b31b0c8c
  Stored in directory: /root/.cache/pip/wheels/c2/8b/ee/728d0fd5aaf2230776060644dd45e2ef

--------

Le but des cellules ci dessous est de créer une fonction parsing_address afin de séparer le nom de la voie ("Avenue") du type de la voie ("du Général De Gaulle").

Nous devons avoir une nomenclature homogène entre l'adresse de BAN et de IPE, c'est pourquoi nous séparons le type de voie du nom de voie.

Une autre possibilité de cette fonction est aussi de repérer les abréviations de type de voie et de les convertir en nom complet (Av --> Avenue). Ces abréviations sont listées dans abbreviations_list, et le nom complet de voie dans fullname_list


In [ ]:
abbreviations_list = [
    'ABE', 'ACH', 'AER', 'AIRE', 'ALL', 'ANSE', 'AR', 'ARC', 'ART', 'AUT', 'AV', 'BAL', 'BAS', 'BASE', 'BAT', 'BCH', 'BCLE', 'BD', 'BER', 'BOIS', 'BRC', 'BRE', 'BRG', 'BRTL', 'BSTD',
    'BUT', 'CALE', 'CAMP', 'CAN', 'CAR', 'CARE', 'CARR', 'CASR', 'CAU', 'CC', 'CCAL', 'CD', 'CEIN', 'CGNE', 'CHE', 'CHEM', 'CHEZ', 'CHI', 'CHL', 'CHP', 'CHS', 'CHT', 'CITE', 'CLOS', 'COL',
    'COLI', 'COR', 'CORO', 'COT', 'COTE', 'COTT', 'COUR', 'CPG', 'CR', 'CRS', 'CST', 'CTR', 'CTRE', 'CVE', 'CHV', 'D', 'DIG', 'DOM', 'DSC', 'ECA', 'ECL', 'EGL', 'EN', 'ENC', 'ENV', 'ESC',
    'ESP', 'ESPA', 'FG', 'FON', 'FORM', 'FORT', 'FOS', 'FRM', 'GAL', 'GARE', 'GBD', 'GPE', 'GPL', 'GPT', 'GR', 'GRI', 'HAM', 'HCH', 'HIP', 'HLE', 'HLG', 'HLM', 'ILE', 'ILOT', 'IMM', 'IMP',
    'JARD', 'JTE', 'LD', 'LEVE', 'LOT', 'MAIL', 'MAIS', 'MAR', 'MAS', 'MLN', 'MTE', 'MUS', 'NTE', 'PAE', 'PAL', 'PARC', 'PAS', 'PASS', 'PAT', 'PAV', 'PCH', 'PERI', 'PIM', 'PISTE', 'PKG', 'PL',
    'PLAN', 'PLCI', 'PLE', 'PLN', 'PLT', 'PN', 'PNT', 'PONT', 'PORT', 'POT', 'PR', 'PRE', 'PROM', 'PRQ', 'PRT', 'PRV', 'PTA', 'PTE', 'PTTE', 'QU', 'QUA', 'R', 'RAC', 'RAID', 'RDE', 'REM', 'RES',
    'RLE', 'RN', 'RNG', 'ROC', 'RPE', 'RPT', 'RTD', 'RTE', 'RUELLETTE', 'RUETTE', 'SEN', 'SENTE', 'SQ', 'STA', 'STDE', 'TOUR', 'TPL', 'TRA', 'TRN', 'TRT', 'TSSE', 'VAL', 'VC', 'VCHE', 'VEN', 'VIA',
    'VLA', 'VLGE', 'VOI', 'VR', 'VTE', 'ZA', 'ZAC', 'ZAD', 'ZI', 'ZONE', 'ZUP'
]
#abbreviations = pd.DataFrame({'Abbreviation': abbreviations_list})

In [ ]:
fullname_list = [
    'Abbaye', 'Ancien chemin', 'Aéroport', 'Aire', 'Allée', 'Anse',
    'Ancienne Rue', 'Arcade', 'Ancienne route', 'Autoroute', 'Avenue', 'Balcon',
    'Bastion', 'Base', 'Bâtiment', 'Bas Chemin', 'Boucle', 'Boulevard', 'Berge',
    'Bois', 'Brèche', 'Barrière', 'Bourg', 'Bretelle', 'Bastide', 'Butte', 'Cale',
    'Camp', 'Canal', 'Carrefour', 'Carrière', 'Carrè', 'Caserne', 'Carreau',
    'Chemin communal', 'Centre Commercial', 'Chemin départemental', 'Ceinture',
    'Campagne', 'Chemin', 'Cheminement', 'Chez', 'Charmille', 'Chalet', 'Chapelle',
    'Chaussée', 'Château', 'Cité', 'Clos', 'Col', 'Colline', 'Corniche', 'Coron', 'Côte',
    'Coteau', 'Cottage', 'Cour', 'Camping', 'Chemin rural', 'Cours', 'Castel',
    'Contour', 'Centre', 'Coursive', 'Chemin Vicinal', 'Départementale', 'Digue',
    'Domaine', 'Descente', 'Écart', 'Écluse', 'Église', 'Enceinte', 'Enclos',
    'Enclave', 'Escalier', 'Esplanade', 'Espace', 'Faubourg', 'Fontaine', 'Forum',
    'Fort', 'Fosse', 'Ferme', 'Galerie', 'Gare', 'Grand boulevard', 'Groupe',
    'Grande Place', 'Groupement', 'Grande rue', 'Grille', 'Hameau', 'Haut Chemin',
    'Hippodrome', 'Halle', 'Halage', 'HLM', 'Île', 'Îlot', 'Immeuble', 'Impasse',
    'Jardin', 'Jetée', 'Lieu dit', 'Levée', 'Lotissement', 'Mail', 'Maison', 'Marché',
    'Mas', 'Moulin', 'Montée', 'Musée', 'Nouvelle Route', 'Petite Avenue', 'Palais',
    'Parc', 'Passage', 'Passe', 'Patio', 'Pavillon', 'Petit Chemin', 'Périphérique',
    'Petite Impasse', 'Piste', 'Parking', 'Place', 'Plan', 'Placis', 'Passerelle',
    'Plaine', 'Plateau', 'Passage à Niveau', 'Pointe', 'Pont', 'Port', 'Poterne',
    'Petite Rue', 'Pré', 'Promenade', "Presqu'île", 'Petite Route', 'Parvis',
     'Petite Allée', 'Porte', 'Placette', 'Quai', 'Quartier', 'Rue', 'Raccourci',
     'Raidillon', 'Ronde', 'Rempart', 'Résidence', 'Ruelle', 'Route Nationale',
    'Rangée', 'Rocade', 'Rampe', 'Rond Point', 'Rotonde', 'Route', 'Ruellette',
    'Ruette', 'Sentier', 'Sente', 'Square', 'Station', 'Stade', 'Tour', 'Terre Plein',
    'Traverse', 'Terrain', 'Tertre', 'Terrasse', 'Vallée', 'Voie Communale',
    'Vieux Chemin', 'Venelle', 'Via', 'Villa', 'Village', 'Voie', 'Vieille Rue',
    'Vieille Route', 'Zone Artisanale', 'Zone Aménagement Concerté',
    'Zone Aménagement Différé', 'Zone Industrielle', 'Zone', 'Zone à Urbaniser en Priorité'
]
fullname_list = [string.lower() for string in fullname_list] # Lower Case the fullname of roadtype
fullname_list = [unidecode(string) for string in fullname_list] # Remove accents from words

#abbreviations = pd.DataFrame({'Abbreviation': abbreviations_list})

In [ ]:
abbreviation_to_type_mapping = dict(zip(abbreviations_list, fullname_list))

In [ ]:
def find_string_contains_multi_words(string_list, nb_words):
  #This function to filter the full name of road type that have more than 1 word
  result_list = []
  for string in string_list:
    words = string.split()
    if len(words) == nb_words:
      result_list.append(string)
  return result_list

find_string_contains_2_words =  find_string_contains_multi_words(fullname_list, 2)
find_string_contains_3_words =  find_string_contains_multi_words(fullname_list, 3)


def compare_2_strings(string_list, words, threshold = 75):
  # Calculate the similarity score using Fuzzy Logic of the selected words with each string of the list
  # We just take into account the score that is higher than threshold
  # The highest score corresponds to the word considered to be the best match in the list
  # If no scores exceed the threshold, return None
  best_match = None
  best_score = 0
  for item in string_list:
        similarity_score = fuzz.ratio(words, item)
        if similarity_score >= threshold and similarity_score > best_score:
          best_match = item
          best_score = similarity_score
  return best_match, best_score

def parsing_address(input_string, words_to_compare = 3):
  # This is the recursive function

  words = input_string.split() #Split the address into list of string

  if len(words) > 1:
    nb_words = min(words_to_compare, len(words) - 1)  #Take into account the number of words to compare
  else:
    return '',input_string
  compared_words =' '.join(words[:nb_words])

  if words[0] == "le" or words[0] == "la": # Address start with le/la easily confuse with ile, villa
    return '', input_string                # So we assign these addresses to Nom de voie

  if nb_words == 1:
  #Create the stop condition for recursive function (the base case)
  # In case we need to compare first 1 word of the address, we will compare it with string in abbreviation list and fullname_list
  # After having compared that word with 2 lists, we will see that the word is more similar to which string in which list (to Return it correctly)
    match_abbre, match_score_abbre = compare_2_strings(abbreviations_list, compared_words.upper()) # Uppercase the word when compare with abbreviation list because the string in this list is uppercased
    match_full , match_score_full = compare_2_strings(fullname_list, unidecode(compared_words.lower())) # Lowercase the word and then remove the accents
    if match_abbre != None or match_full != None:
      if match_score_abbre > match_score_full:
        return abbreviation_to_type_mapping[match_abbre], ' '.join(words[1:])
      else:
        return match_full,  ' '.join(words[1:])
    else:
      return '', input_string # If no string in both lists matches that word then Invalid Address or there 's no type of the road in this address

  else:
     # If type of the roads contain 2 or 3 words then
     # we need to compare the first 2,3 words of the input string with the string in fullname_list
     # If Match, stop the function immediately
     # If not, decrease the nb of words for comparison from 3 to 2 or 2 to 1.
    reference_list = find_string_contains_2_words if nb_words == 2 else find_string_contains_3_words
    match_word, match_score = compare_2_strings(reference_list, unidecode(compared_words.lower()))
    if match_word == None :
      return parsing_address(input_string, nb_words - 1)
    else:
      return match_word,  input_string.replace(compared_words,"")

Ainsi, la fonction parsing_address prend en entrée un élément de la colonne "nom_voie" de BAN et le convertit en élément de "TypeVoieImmeuble" et "NomVoieImmeuble" de IPE.

---------

# Preprocessing

Dans cette partie, nous allons devoir obtenir un schéma de données similaire entre la table BAN et la table IPE.
Ainsi, nous devons avoir les mêmes colonnes (notamment séparer le type de voie du nom de voie dans BAN) ainsi qu'avoir des noms de colonnes identiques.

De plus, nous devons nous assurer que le contenu des données est similaires, et donc traiter les majuscules, accents, cartactères spéciaux.

In [ ]:
ipe = pd.read_csv('/content/IPE-AXTD.csv', delimiter=";")
ban = pd.read_csv('/content/BAN-adresses-12.csv', delimiter=";")
# we take ban-adresses-12 as it is the department 12, which is contianed in ipe AXTD region

In [ ]:
# ban['TypeVoie'] = ban['nom_voie'].apply(lambda x : parsing_address(x)[0])
# ban['NomVoieReste'] = ban['nom_voie'].apply(lambda x : parsing_address(x)[1])  #We shouldn't write like this because it will double the processing time

for i in range(len(ban)):
    parsed_address = parsing_address(ban.loc[i, 'nom_voie'])
    #print(i)
    #if parsed_address is not None:
    ban.loc[i, 'TypeVoie'] = parsed_address[0]
    ban.loc[i, 'NomVoieReste'] = parsed_address[1]
ban

,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,x,y,lon,lat,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles,TypeVoie,NomVoieReste
0,12002_0puwwe_00001,NaN,1,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707758.86,6339740.77,3.097003,44.156631,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
1,12002_0puwwe_00003,NaN,3,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707754.93,6339752.98,3.096954,44.156741,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
2,12002_0puwwe_00005,NaN,5,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707758.99,6339764.54,3.097005,44.156845,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
3,12002_0puwwe_00007,NaN,7,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707766.91,6339767.00,3.097104,44.156867,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
4,12002_0puwwe_00009,NaN,9,NaN,Plan du Foirail,12520,12002,Aguessac,NaN,NaN,707763.40,6339758.66,3.097060,44.156792,entrée,NaN,NaN,AGUESSAC,PLAN DU FOIRAIL,inconnue,inconnue,0,NaN,plan,du Foirail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152678,12006_b167_00018,12006_B167,18,NaN,Mas Nespoulous,12430,12006,Alrance,NaN,NaN,675581.14,6334943.59,2.694943,44.113072,entrée,NaN,NaN,ALRANCE,MAS NESPOULOUS,commune,commune,1,120060000E0619,mas,Nespoulous
152679,12006_b167_00020,12006_B167,20,NaN,Mas Nespoulous,12430,12006,Alrance,NaN,NaN,675674.74,6334846.90,2.696117,44.112205,entrée,NaN,NaN,ALRANCE,MAS NESPOULOUS,commune,commune,1,120060000E0743,mas,Nespoulous
152680,12006_b002_00001,12006_B002,1,NaN,La Baraque de Malet,12430,12006,Alrance,NaN,NaN,675308.07,6334466.23,2.691555,44.108766,entrée,NaN,NaN,ALRANCE,LA BARAQUE DE MALET,commune,commune,1,120060000F0538,villa,Baraque de Malet
152681,12006_b002_00003,12006_B002,3,NaN,La Baraque de Malet,12430,12006,Alrance,NaN,NaN,675282.59,6334438.89,2.691238,44.108519,entrée,NaN,NaN,ALRANCE,LA BARAQUE DE MALET,commune,commune,1,120060000F0537,villa,Baraque de Malet


In [ ]:
ban.rename(columns={'typeVoie': 'TypeVoie'}, inplace=True)
ban.rename(columns={'nomVoie': 'NomVoieReste'}, inplace=True)

ban_reduced = ban[['id','numero','rep','TypeVoie','NomVoieReste','code_postal','nom_commune','x','y']]
ipe_reduced = ipe[['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']]
ipe_reduced['TypeVoieImmeuble'] = ipe_reduced['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')


nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
ban_reduced = ban_reduced.rename(columns=nom_colonnes)

In [ ]:
ban_reduced

,IdentifiantImmeuble,NumeroVoieImmeuble,ComplementNumeroVoieImmeuble,TypeVoieImmeuble,NomVoieImmeuble,CodePostalImmeuble,CommuneImmeuble,CoordonneeImmeubleX,CoordonneeImmeubleY
0,12002_0puwwe_00001,1,NaN,plan,du Foirail,12520,Aguessac,707758.86,6339740.77
1,12002_0puwwe_00003,3,NaN,plan,du Foirail,12520,Aguessac,707754.93,6339752.98
2,12002_0puwwe_00005,5,NaN,plan,du Foirail,12520,Aguessac,707758.99,6339764.54
3,12002_0puwwe_00007,7,NaN,plan,du Foirail,12520,Aguessac,707766.91,6339767.00
4,12002_0puwwe_00009,9,NaN,plan,du Foirail,12520,Aguessac,707763.40,6339758.66
...,...,...,...,...,...,...,...,...,...
152678,12006_b167_00018,18,NaN,mas,Nespoulous,12430,Alrance,675581.14,6334943.59
152679,12006_b167_00020,20,NaN,mas,Nespoulous,12430,Alrance,675674.74,6334846.90
152680,12006_b002_00001,1,NaN,villa,Baraque de Malet,12430,Alrance,675308.07,6334466.23
152681,12006_b002_00003,3,NaN,villa,Baraque de Malet,12430,Alrance,675282.59,6334438.89


In [ ]:
ipe_reduced

,IdentifiantImmeuble,NumeroVoieImmeuble,ComplementNumeroVoieImmeuble,TypeVoieImmeuble,NomVoieImmeuble,CodePostalImmeuble,CommuneImmeuble,CoordonneeImmeubleX,CoordonneeImmeubleY
0,IMB/12001/X/0001,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.099759,6.361511e+06
1,IMB/12001/X/0002,8,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674437.822388,6.361478e+06
2,IMB/12001/X/0003,6,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674408.962423,6.361491e+06
3,IMB/12001/X/0004,10,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674402.615772,6.361449e+06
4,IMB/12001/X/0005,1,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674523.068416,6.361517e+06
...,...,...,...,...,...,...,...,...,...
283826,IMB/48198/X/00TG,5996,NaN,route,du Mas de la Barque,48800,Villefort,773209.199944,6.371295e+06
283827,IMB/48198/X/00TH,6005,NaN,route,du Mas de la Barque,48800,Villefort,773186.659129,6.371244e+06
283828,IMB/48198/X/00TI,5264,NaN,route,du Mas de la Barque,48800,Villefort,773131.114161,6.371255e+06
283829,IMB/48198/X/00TJ,6033,NaN,route,du Mas de la Barque,48800,Villefort,773114.651880,6.371262e+06



Ne pas éxécuter cette section à chaque fois, car elle prend beaucoup de temps d'exécution. L'utilisation optimale est de traiter une fois les données brutes de BAN et IPE et ensuite d'enregister les données traitées ban_reduced & ipe_reduced dans des CSV pour les réutiliser plus tard si besoin.

In [ ]:
ipe_reduced.to_csv('/content/ipe_reduced.csv')
ban_reduced.to_csv('/content/ban_reduced.csv')

------------

In [ ]:
"""
ipe = pd.read_csv('/content/IPE-AXTD.csv', delimiter=";")
ban_reduced = pd.read_csv('/content/ban_decomposed-12.csv')

ban_reduced = ban_reduced[['id','numero','rep','TypeVoie','NomVoieReste','code_postal','nom_commune','x','y']]
nom_colonnes = {'id': 'IdentifiantImmeuble','numero' : 'NumeroVoieImmeuble','rep' : 'ComplementNumeroVoieImmeuble','TypeVoie' : 'TypeVoieImmeuble','NomVoieReste' : 'NomVoieImmeuble','code_postal' : 'CodePostalImmeuble','nom_commune' : 'CommuneImmeuble','x' : 'CoordonneeImmeubleX','y' : 'CoordonneeImmeubleY'}
ban_reduced = ban_reduced.rename(columns=nom_colonnes)

ipe_reduced = ipe[['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']]
ipe_reduced['TypeVoieImmeuble'] = ipe_reduced['TypeVoieImmeuble'].apply(lambda x : unidecode(x).lower() if pd.isna(x) is False else '')
"""

#si vous avez enregistré les dataframes de la section précédente sous forme de CSV pour les réutiliser, vous pouvez les charger ici en précisant le chemin des CSV.
ban_reduced = pd.read_csv('/content/ban_reduced.csv')
ipe_reduced = pd.read_csv('/content/ipe_reduced.csv')


--------

# Adresses similaires

Dans cette section, nous utilisons la bibliothèques py_entitymatching afin de trouver les adresses similaires entre les 2 bases de données BAN et IPE.


In [ ]:
A = ipe_reduced.copy()
B = ban_reduced.copy()

em.set_key(A, 'IdentifiantImmeuble')
em.set_key(B, 'IdentifiantImmeuble')


shape_A = A.shape
shape_B = B.shape
print(shape_A)
print(shape_B)

(283831, 9)
(152683, 9)


In [ ]:
def complement_numero_voie(string) :
  try :
    string = string.lower()
  except :
    pass
  if string == 'bis' :
    string = 'b'
  if string == 'ter' :
    string = 't'
  if string == 'quater' :
    string = 'q'
  return string

A['ComplementNumeroVoieImmeuble'] = A['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)
B['ComplementNumeroVoieImmeuble'] = B['ComplementNumeroVoieImmeuble'].apply(complement_numero_voie)


### Etape 1 : Blocking

ETAPE 1

On compare certaines colonnes de manière stricte --> technique de **blocking**


C'est à dire que s'il n'y a pas de correspondance stricte sur chacune de ces colonnes entre les 2 adresses comparées, on considère que ce ne sont pas les mêmes adresses.

Les colonnes blockées sont les suivantes :
- **NumeroVoieImmeuble**
- **ComplementNumeroVoieImmeuble**
- **TypeVoieImmeuble**
- **CodePostalImmeuble**

In [ ]:
# define the blocker attributes to check and put in 1 column (because AttrEquivalenceBlocker can only handle 1 column comparison)
# for the unknown NumeroVoieImmeuble, we impute with the value 0
A['NumeroVoieImmeuble'] = A['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(A[A['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the IPE dataset')

B['NumeroVoieImmeuble'] = B['NumeroVoieImmeuble'].fillna(value = 0).astype(int)
print('We imputed', len(B[B['NumeroVoieImmeuble'] == 0]), 'unknown NumeroVoieImmeuble in the BAN dataset')

A['CodePostalImmeuble'] = A['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(A[A['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the IPE dataset')

B['CodePostalImmeuble'] = B['CodePostalImmeuble'].fillna(value = 00000).astype(int)
print('We imputed', len(B[B['CodePostalImmeuble'] == 0]), 'unknown CodePostalImmeuble in the BAN dataset')




A['blocking_data'] = A['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  A['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + A['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + A['CodePostalImmeuble'].fillna('').astype(str)
B['blocking_data'] = B['NumeroVoieImmeuble'].fillna('').astype(str)+ ' '  +  B['ComplementNumeroVoieImmeuble'].fillna('').astype(str) + ' ' + B['TypeVoieImmeuble'].fillna('').astype(str) + ' ' + B['CodePostalImmeuble'].fillna('').astype(str)

We imputed 122228 unknown NumeroVoieImmeuble in the IPE dataset
We imputed 10 unknown NumeroVoieImmeuble in the BAN dataset
We imputed 0 unknown CodePostalImmeuble in the IPE dataset
We imputed 0 unknown CodePostalImmeuble in the BAN dataset


In [ ]:
attributes = ['IdentifiantImmeuble','NumeroVoieImmeuble','ComplementNumeroVoieImmeuble','TypeVoieImmeuble','NomVoieImmeuble','CodePostalImmeuble','CommuneImmeuble','CoordonneeImmeubleX','CoordonneeImmeubleY']

ab = em.AttrEquivalenceBlocker()

# Use block_tables to apply blocking over two input tables.
C = ab.block_tables(A, B,
                    l_block_attr='blocking_data', r_block_attr='blocking_data',
                    l_output_attrs=attributes,
                    r_output_attrs=attributes,
                    l_output_prefix='l_', r_output_prefix='r_')
C

,_id,l_IdentifiantImmeuble,r_IdentifiantImmeuble,l_NumeroVoieImmeuble,l_ComplementNumeroVoieImmeuble,l_TypeVoieImmeuble,l_NomVoieImmeuble,l_CodePostalImmeuble,l_CommuneImmeuble,l_CoordonneeImmeubleX,l_CoordonneeImmeubleY,r_NumeroVoieImmeuble,r_ComplementNumeroVoieImmeuble,r_TypeVoieImmeuble,r_NomVoieImmeuble,r_CodePostalImmeuble,r_CommuneImmeuble,r_CoordonneeImmeubleX,r_CoordonneeImmeubleY
0,0,IMB/12001/X/0001,12001_0052_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,Adrien Vezinhet,12630,Agen-d'Aveyron,673832.68,6362135.56
1,1,IMB/12001/X/0001,12001_0040_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,Daude de Prades,12630,Agen-d'Aveyron,673699.44,6362045.91
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82
3,3,IMB/12001/X/0001,12001_0067_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52
4,4,IMB/12001/X/0001,12001_0069_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Croux,12630,Agen-d'Aveyron,674318.84,6362009.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.7426,6344054.694,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.2682,6344332.362,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.5716,6344372.058,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.0282,6347025.643,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43


Le dataframe affiché contient toutes les adresses "candidates" qui ont passé l'étape du blocking (étape 1)

### Etape 2 : Filtre distance

**ETAPE 2**

Nous utilisons les coordonnées pour calculer la distance entre 2 adresses. Si la distance est supérieure à 2000m (on peut augmenter/baisser cette limite), on considère que ce ne sont pas les mêmes adresses car trop éloignées.

In [ ]:
def distance_calculator(row):
    # left, right will be of type pandas series

    # get coordinates X Y
    left_x = row['l_CoordonneeImmeubleX']
    left_y = row['l_CoordonneeImmeubleY']
    right_x = row['r_CoordonneeImmeubleX']
    right_y = row['r_CoordonneeImmeubleY']

    distance = math.sqrt((right_x - left_x)**2 + (right_y - left_y)**2)

    return distance


In [ ]:
print('old shape', C.shape)
C['distanceXY'] = C[['l_CoordonneeImmeubleX','l_CoordonneeImmeubleY','r_CoordonneeImmeubleX','r_CoordonneeImmeubleY']].apply(lambda row : distance_calculator(row)  ,axis = 1)

# we only take rows where distance <= 2000

C = C[C['distanceXY'] <= 2000]

print('new shape after distance filtering', C.shape)
C

old shape (1133865, 19)
new shape after distance filtering (555978, 20)


,_id,l_IdentifiantImmeuble,r_IdentifiantImmeuble,l_NumeroVoieImmeuble,l_ComplementNumeroVoieImmeuble,l_TypeVoieImmeuble,l_NomVoieImmeuble,l_CodePostalImmeuble,l_CommuneImmeuble,l_CoordonneeImmeubleX,l_CoordonneeImmeubleY,r_NumeroVoieImmeuble,r_ComplementNumeroVoieImmeuble,r_TypeVoieImmeuble,r_NomVoieImmeuble,r_CodePostalImmeuble,r_CommuneImmeuble,r_CoordonneeImmeubleX,r_CoordonneeImmeubleY,distanceXY
0,0,IMB/12001/X/0001,12001_0052_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,Adrien Vezinhet,12630,Agen-d'Aveyron,673832.68,6362135.56,869.844563
1,1,IMB/12001/X/0001,12001_0040_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,Daude de Prades,12630,Agen-d'Aveyron,673699.44,6362045.91,912.011620
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82,33.955654
4,4,IMB/12001/X/0001,12001_0069_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Croux,12630,Agen-d'Aveyron,674318.84,6362009.03,512.126658
5,5,IMB/12001/X/0001,12001_0072_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Parro,12630,Agen-d'Aveyron,674961.77,6361435.86,529.031137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.7426,6344054.694,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54,40.512861
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.2682,6344332.362,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61,44.155615
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.5716,6344372.058,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61,47.087184
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.0282,6347025.643,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43,18.890861


### Etape 3 : Matching

**ETAPE 3**

On calcule des cores de similarité basé sur la distance de Levensthein.
On calcule sur les colonnes qui peuvent avoir des sources d'erreurs plus complexes, comme des fautes de frappes, abréviations, etc... --> technique de **matching**

- **CommuneImmeuble**
- **NomVoieImmeuble**

Si le score de similarité est supérieure à **70%** pour la communne et **80%** pour le nom de la voie, les adresses sont considérées comme similaires.

In [ ]:
def jaccard_similarity_score(string1, string2):
  alphabet_tok = sm.AlphabeticTokenizer()
  token1 = alphabet_tok.tokenize(string1)
  token2 = alphabet_tok.tokenize(string2)
  jaccard = sm.Jaccard()  # Example Jaccard similarity
  return jaccard.get_sim_score(token1, token2)

def levenshtein_similarity_score(string1, string2):
  distance = sm.Levenshtein()
  return distance.get_sim_score(string1.lower().replace('-',' '), string2.lower().replace('-',' '))

# Apply the function to create a new column 'SimilarityScore'
C['SimilarityScoreRue'] = C.apply(lambda x : levenshtein_similarity_score(x['l_NomVoieImmeuble'],x['r_NomVoieImmeuble']), axis=1)
C['SimilarityScoreCommune'] = C.apply(lambda x : levenshtein_similarity_score(x['l_CommuneImmeuble'],x['r_CommuneImmeuble']), axis=1)


In [ ]:
#first, we filter the commune similiarity
C_filtered = C[C['SimilarityScoreCommune'] >= 0.70]
#then, we filter the street name similiarity
C_filtered = C_filtered[C_filtered['SimilarityScoreRue'] >= 0.80]
print('From table A : ' ,shape_A[0] , '    rows and table B : ', shape_B[0], '   rows we found ', len(C_filtered) , ' matching adresses')

C_filtered

From table A :  283831     rows and table B :  152683    rows we found  74578  matching adresses


,_id,l_IdentifiantImmeuble,r_IdentifiantImmeuble,l_NumeroVoieImmeuble,l_ComplementNumeroVoieImmeuble,l_TypeVoieImmeuble,l_NomVoieImmeuble,l_CodePostalImmeuble,l_CommuneImmeuble,l_CoordonneeImmeubleX,l_CoordonneeImmeubleY,r_NumeroVoieImmeuble,r_ComplementNumeroVoieImmeuble,r_TypeVoieImmeuble,r_NomVoieImmeuble,r_CodePostalImmeuble,r_CommuneImmeuble,r_CoordonneeImmeubleX,r_CoordonneeImmeubleY,distanceXY,SimilarityScoreRue,SimilarityScoreCommune
2,2,IMB/12001/X/0001,12001_0083_00004,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674438.0998,6361510.983,4,NaN,impasse,de la Baraque Basse,12630,Agen-d'Aveyron,674471.55,6361516.82,33.955654,1.0,1.0
18,18,IMB/12001/X/0023,12001_0072_00004,4,NaN,impasse,de la Parro,12630,Agen-d'Aveyron,674996.8645,6361415.831,4,NaN,impasse,de la Parro,12630,Agen-d'Aveyron,674961.77,6361435.86,40.407732,1.0,1.0
34,34,IMB/12001/X/004Z,12001_0071_00004,4,NaN,impasse,des Palanges,12630,Agen-d'Aveyron,677296.5915,6360515.681,4,NaN,impasse,des Palanges,12630,Agen-d'Aveyron,677294.97,6360514.05,2.299875,1.0,1.0
42,42,IMB/12001/X/0052,12001_0067_00004,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677162.7574,6360389.044,4,NaN,impasse,de la Cascade,12630,Agen-d'Aveyron,677171.63,6360381.52,11.633297,1.0,1.0
59,59,IMB/12001/X/005R,12001_0070_00004,4,NaN,impasse,des Lignes,12630,Agen-d'Aveyron,672674.9673,6361881.464,4,NaN,impasse,des Lignes,12630,Agen-d'Aveyron,672659.70,6361882.59,15.308766,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133860,1133860,IMB/12307/X/00BB,12307_0018_00408,408,NaN,chemin,de Bellevue,12410,Curan,689708.7426,6344054.694,408,NaN,chemin,de Bellevue,12410,Curan,689727.62,6344090.54,40.512861,1.0,1.0
1133861,1133861,IMB/12307/X/00BC,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688529.2682,6344332.362,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61,44.155615,1.0,1.0
1133862,1133862,IMB/12307/X/00BG,12307_0039_00064,64,NaN,impasse,du Claux,12410,Curan,688560.5716,6344372.058,64,NaN,impasse,du Claux,12410,Curan,688513.51,6344373.61,47.087184,1.0,1.0
1133863,1133863,IMB/12307/X/00BH,12307_0084_00349,349,NaN,chemin,de Martials,12410,Curan,689480.0282,6347025.643,349,NaN,chemin,de Martials,12410,Curan,689478.05,6347044.43,18.890861,1.0,1.0


In [ ]:
print('We found', len(C_filtered), 'matching adresses')

We found 74578 matching adresses


Ce dataframe C_filtered contient uniquement les adresses de IPE ( à gauche ) et BAN ( à droite ) qui sont considérées comme similaires, avec les score de similarité associés, c'est à dire la confiance avec laquelle on peut affirmer ces résultats.

In [ ]:
C_filtered.to_csv('matched_table_BAN_IPE_UsingRulesBase.csv')